# Info

This notebook consists of methods I've used to parse and visualize different polygons and coordinates. Parameters need to be set accordingly.

# Imports

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np

In [ ]:
from bokeh.tile_providers import OSM, get_provider
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource
from bokeh.models import GeoJSONDataSource

In [ ]:
from shapely.geometry import Point
from shapely import wkt

# Parse data functions

In [1]:
def load_polygon_geojson(file_path: str):
    return gpd.read_file(file_path)

def load_polygon_excel(file_path: str, crs: str='EPSG:4326', epsg: int=3857):
    """

    :param file_path: path to the excel file that has polygon coordinates
    :param crs: needs to be set based on use case
    :param epsg: needs to be set based on use case
    :return: geopandas dataframe with polygons in 'geometry' column
    """
    polygon_df = pd.read_excel(file_path)
    polygon_df = polygon_df.pipe(gpd.GeoDataFrame)

    polygon_df['geometry'] = polygon_df['geometry'].apply(wkt.loads)
    gdf = gpd.GeoDataFrame(polygon_df)
    gdf.crs = crs
    gdf = gdf.to_crs(epsg=epsg)

    return gdf


def load_polygon_wkt_csv(file_path: str, crs: str='EPSG:4326', epsg: int=3857):
    def tryParseWKT(x):
        try:
            return wkt.loads(x)
        except:
            return None

    polygon_df = pd.read_csv(file_path)

    polygon_df['geometry'] = polygon_df['geometry'].apply(lambda x: tryParseWKT(x))
    gdf = gpd.GeoDataFrame(polygon_df[polygon_df['geometry'].notna()], crs=crs)

    return gdf.to_crs(epsg=epsg)


In [ ]:
def to_decimal(S):
    """
    convert degrees to lat or long
    :param S: input str
    :return: numpy array of parsed coordinates
    """
    S = str(S)
    try:
        if '°' not in S:
            return float(S)
        S = S.split('"')[0]
        S = float(S.split('°')[0]) + float(S.split('°')[1].split("'")[0])/60 + float(S.split('°')[1].split("'")[1])/3600
        return S
    except:
        return np.nan

df = pd.read_excel("sample.xlsx")

df['Lat'] = df['t'].apply(lambda x: to_decimal(x))
df['Long'] = df['g'].transform(lambda x: to_decimal(x))

# Map functions

In [2]:
def in_polygon(lat, lng, gdf):
    """
    check if a point is inside a polygon.
    return the first match.
    the gdf is geopandas dataframe.
    """
    point = Point(lng, lat)
    for i, row in gdf.iterrows():
        polygon = row['area_geometry']
        if point.within(polygon):
            return row

In [ ]:
def merc_from_arrays(lats, lons):
    r_major = 6378137.000
    x = r_major * np.radians(lons)
    scale = x/lons
    y = 180.0/np.pi * np.log(np.tan(np.pi/4.0 + lats * (np.pi/180.0)/2.0)) * scale
    return (x, y)

In [ ]:
import json

def get_geodatasource(df):
    """Get getjsondatasource from geopandas object"""
    json_data = json.dumps(json.loads(df.to_json()))
    return GeoJSONDataSource(geojson = json_data)

# Generate maps

In [ ]:
### READ Area Polygon
gdf = load_polygon_excel('areas.xlsx')

In [ ]:
### READ FILE HERE. COLUMN NAMES: StoreID, Lat, Long
df = pd.read_excel('sample.xlsx')

In [ ]:
tile_provider = get_provider(OSM)

p = figure(x_range=(4855769, 7080637), y_range=(1644191, 4744721), sizing_mode='scale_both', 
           tools="pan,wheel_zoom,crosshair,reset,save", active_scroll="wheel_zoom",
           tooltips=[('StoreID', "@StoreID"),
                    ('StoreID', "@new"),
                    ('StoreID', "@StoreID")])

p.add_tile(tile_provider)

In [ ]:
longitude, latitude = merc_from_arrays(np.array(df['Lat'].astype(float)), np.array(df['Long'].astype(float)))

source = ColumnDataSource(data=dict(longitude=longitude, latitude=latitude, StoreID=np.array(df['StoreID'])))
p.scatter(x='longitude', y='latitude', radius=50, fill_alpha=0.7,
          fill_color='Blue', source=source)

source = ColumnDataSource(data=dict(longitude=longitude, latitude=latitude, StoreID=np.array(df['StoreID'])))
p.scatter(x='longitude', y='latitude', radius=3500, fill_alpha=0.3,
          fill_color='Blue', source=source)

In [ ]:
########### Add areas
p.patches('xs','ys', source=get_geodatasource(gdf),
          fill_alpha=0.8, color = 'grey',
          line_color="black", line_width=0.2)

In [ ]:
output_file('lat_long.html')

show(p)